In [1]:
import json
from collections import Counter

import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

### [Question 1](#Question 1)

What are the top 3 and the bottom 3 states in terms of number of users?

In [6]:
with open('song.json') as jsonfile:
    data = json.load(jsonfile)


In [27]:
df = pd.DataFrame(data)
df.head()
len(df['user_id'].unique())
len(df)

4000

In [50]:
len(df.loc[df['user_state'] == 'New York']['user_id'].unique()
)

23

In [46]:
df.head()

,id,song_played,time_played,user_id,user_sign_up_date,user_state
0,GOQMMKSQQH,Hey Jude,2015-06-11 21:51:35,122,2015-05-16,Louisiana
1,HWKKBQKNWI,We Can Work It Out,2015-06-06 16:49:19,3,2015-05-01,Ohio
2,DKQSXVNJDH,Back In the U.S.S.R.,2015-06-14 02:11:29,35,2015-05-04,New Jersey
3,HLHRIDQTUW,P.s. I Love You,2015-06-08 12:26:10,126,2015-05-16,Illinois
4,SUKJCSBCYW,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00,6,2015-05-01,New Jersey


In [79]:
df_state = df.groupby(['user_state','user_id']).size().reset_index(name='user_counts')

df_state.groupby('user_state').count().sort_values(by ='user_id', ascending = False).head()

,user_id,user_counts
user_state,,
New York,23,23
California,21,21
Texas,15,15
Pennsylvania,9,9
Ohio,9,9


<a id='Question 1'></a>
### Answer Q1
The top three states by count are-  New York, California, Texas
The bottom three states by count are - Nebraska, Arizona, Kansas

### [Question 2](#Question 2)

What are the top 3 and the bottom 3 states in terms of user engagement? You can choose how to mathematically deﬁne user engagement. What the CEO cares about here is in which states users are using the product a lot/very little

I think the user_engagment can be defined as how many times the user has listened to music

In [123]:
df['time_played_date'] = pd.to_datetime(df['time_played']).apply(lambda x: x.date())
df['user_sign_up_date'] = pd.to_datetime(df['user_sign_up_date']).apply(lambda x: x.date())
df['active'] = df['time_played_date'] - df['user_sign_up_date']
df['active'] = df['active'] / np.timedelta64(1, 'h')/24

In [137]:
df_date=df.groupby(['user_state','user_id']).size().reset_index(name='date_counts')

In [165]:
df_rank = df_date.groupby('user_state').count().drop(['user_id'], axis = 1)
# We take the mean of the number of times as song has been played per state per user 

In [166]:
#The more active mean, the more time user takes to play a song since he registered.
df_rank['active_mean'] = df.groupby(['user_state'])['active'].mean()
df_rank['active_mean'] = df_rank['active_mean'].apply(lambda x:1/x)

In [170]:
df_rank.sort_values(by = ['date_counts', 'active_mean'], ascending= False).head()

,date_counts,active_mean
user_state,,
New York,23,0.028899
California,21,0.030359
Texas,15,0.029609
Pennsylvania,9,0.029301
Ohio,9,0.026256


### [Question 3](#Question 3)
Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next. That is, if, for instance, a user is currently listening to "Eight Days A Week", which song has the highest probability of being played right after it by the same user? This is going to be v1 of a song recommendation model.
We try following recommendation systems -
* [Item-Item based RS](#Item)


In [187]:
df_song = df[['user_id','song_played','id']]

In [188]:
df_song = df_song.groupby(['song_played','user_id']).size().reset_index(name='song_counts')

In [194]:
print("No of unique users",len(df['user_id'].unique()))
## return 196 unique users
print("No of unique songs",len(df['song_played'].unique()))
## return 100 unique songs

No of unique users 196
No of unique songs 100


In [196]:
df_song['song_counts'].describe()

count    2772.000000
mean        1.443001
std         0.887784
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        10.000000
Name: song_counts, dtype: float64

Most of the songs are listened once (median is 1)

In [197]:
# Convert to binary data
df_song.groupy()

,song_played,user_id,song_counts
0,A Day In The Life,3,1
1,A Day In The Life,4,3
2,A Day In The Life,6,2
3,A Day In The Life,12,2
4,A Day In The Life,13,3


In [213]:
df_reshape = df_song.pivot(index='user_id', columns='song_played', values='song_counts')
df_reshape = df_reshape.fillna(0)

In [214]:
df_reshape.head()

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,BALLAD OF JOHN AND YOKO,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


<a id='Item'></a>
#### Item- based RS- 
Reminder: In item based collaborative filtering we do not care about the user column.

In [225]:
# PLaceholder
data_ibs = pd.DataFrame(index=df_reshape.columns,columns=df_reshape.columns)

In [226]:
# Fill in those empty spaces with cosine similarities
from scipy.spatial.distance import cosine
for i in range(0,len(data_ibs.columns)) :
    for j in range(0,len(data_ibs.columns)) :
        data_ibs.ix[i,j] = 1-cosine(df_reshape.iloc[:,i],df_reshape.iloc[:,j])

c:\anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  """


In [227]:
data_ibs.head()

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,BALLAD OF JOHN AND YOKO,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,1,0.235702,0.0745356,0.119523,0.212132,0.355023,0.329404,0.152145,0.210819,0.172133,...,0.464938,0.030429,0.508964,0.223607,0.359092,0.0372678,0.318198,0.35322,0.087841,0
A Hard Day's Night,0.235702,1,0,0,0.1,0.136931,0.111803,0,0,0.0912871,...,0.259548,0.129099,0.210099,0,0,0,0.05,0.195468,0.0745356,0
A Saturday Club Xmas/Crimble Medley,0.0745356,0,1,0,0,0,0,0,0,0,...,0.109435,0,0,0.166667,0,0,0,0,0,0
ANYTIME AT ALL,0.119523,0,0,1,0,0.154303,0.0944911,0.109109,0,0,...,0.116991,0,0.138107,0.0890871,0.183942,0,0,0.146845,0,0
Across The Universe,0.212132,0.1,0,0,1,0.0912871,0,0,0,0,...,0.138426,0,0.116722,0,0,0,0,0.0434372,0,0


In [231]:
# Create a placeholder items for closes neighbours to an item
data_neighbours = pd.DataFrame(index=data_ibs.columns,columns=range(1,11))

# Loop through our similarity dataframe and fill in neighbouring item names
for i in range(0,len(data_ibs.columns)):
    data_neighbours.iloc[i,:10] = data_ibs.iloc[0:,i].sort_values(ascending=False)[:10].index

# --- End Item Based Recommendations --- #

In [232]:
data_neighbours

,1,2,3,4,5,6,7,8,9,10
song_played,,,,,,,,,,
A Day In The Life,A Day In The Life,Come Together,Revolution,Get Back,While My Guitar Gently Weeps,Back In the U.S.S.R.,Hey Jude,Let It Be,Here Comes The Sun,Lucy In The Sky With Diamonds
A Hard Day's Night,A Hard Day's Night,Come Together,Let It Be,Back In the U.S.S.R.,Here Comes The Sun,Revolution,Sun King,Hey Jude,We Can Work It Out,Get Back
A Saturday Club Xmas/Crimble Medley,A Saturday Club Xmas/Crimble Medley,GIRL,IT WON'T BE LONG,Hey Jude,I Want You (She's So Heavy),Back In the U.S.S.R.,Wild Honey Pie,Twist and Shout,Golden Slumbers,We Can Work It Out
ANYTIME AT ALL,ANYTIME AT ALL,Can't Buy Me Love,Come Together,Back In the U.S.S.R.,Helter Skelter,LITTLE CHILD,GIRL,Love Me Do,Let It Be,With a Little Help From My Friends
Across The Universe,Across The Universe,Revolution,Birthday,LITTLE CHILD,Love Me Do,A Day In The Life,Strawberry Fields Forever,Reprise / Day in the Life,Penny Lane,IN MY LIFE
All My Loving,All My Loving,Let It Be,Hey Jude,Come Together,A Day In The Life,And Your Bird Can Sing,Lucy In The Sky With Diamonds,Eight Days A Week,Reprise / Day in the Life,While My Guitar Gently Weeps
All You Need Is Love,All You Need Is Love,A Day In The Life,While My Guitar Gently Weeps,Revolution,Yesterday,Come Together,I Am the Walrus,Get Back,Lucy In The Sky With Diamonds,Norwegian Wood
And Your Bird Can Sing,And Your Bird Can Sing,All My Loving,IN MY LIFE,SHE'S A WOMAN,The Long And Winding Road,Golden Slumbers,Yellow Submarine,Come Together,Let It Be,Penny Lane
BAD BOY,BAD BOY,Hey Jude,OH DARLING,A Day In The Life,Fixing A Hole,I Am the Walrus,Let It Be,Sgt. Pepper/with A Little Help From My Friends,And Your Bird Can Sing,When I'm 64


In [233]:
data_ibs.iloc[0:,i]

song_played
A Day In The Life                                                  0
A Hard Day's Night                                                 0
A Saturday Club Xmas/Crimble Medley                                0
ANYTIME AT ALL                                                     0
Across The Universe                                                0
All My Loving                                                      0
All You Need Is Love                                               0
And Your Bird Can Sing                                             0
BAD BOY                                                            0
BALLAD OF JOHN AND YOKO                                            0
Baby You're A Rich Man                                             0
Back In the U.S.S.R.                                       0.0391031
Birthday                                                   0.0642824
Blackbird                                                          0
Can't Buy Me Love     